In [1]:
import time
import re
import os
import platform
# import winsound
import pandas as pd

import pyautogui

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from random import *
from pathlib import Path
from dateutil import parser

from datetime import datetime
import time

In [2]:
def set_chrome_driver():
    os_name = platform.system()

    executable_path=""
    if os_name == "Darwin":
        executable_path=r'./chromedriver'
    elif os_name == "Windows":
        executable_path=r'c:/temp/chromedriver.exe'
    service = Service(executable_path)
    userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)s Chrome/92.0.4515.131 Safari/537.36"
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={userAgent}")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--lang=kr')  # set your language here
    driver = webdriver.Chrome(service=service, options=options)

    # driver.set_window_size(1400, 1040)
    
    return driver


def set_translate_to_kr(driver):
    actionChains = ActionChains(driver)
    actionChains.context_click().perform()

    # here maybe problem. Debug it:
    for i in range(3):
        pyautogui.press('up')

    pyautogui.press('enter')
    pyautogui.sleep(1)

def scroll_page(driver):
    # Set the interval between scrolls in seconds
    scroll_interval = 1

    # Get the initial height of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Send the down arrow key to scroll down the page
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        
        # Wait for the specified interval
        time.sleep(scroll_interval)
        
        # Calculate the new height of the page after scrolling
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Check if the page has reached the end
        if new_height == last_height:
            break
        
        # Update the previous height
        last_height = new_height

def save_page(driver, fname):
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    with open(fname, "w", encoding='utf-8') as file:
        file.write('<!DOCTYPE html>\n' + str(bs))

In [3]:
def sleep_random_sec(sec):
    for t in reversed(range(randint(1, sec))):
        print(str(t), end='', flush=True)
        time.sleep(1)
    print('\r', end='')

def getExamData(driver, dataid):
    url = "https://www.examtopics.com/ajax/discussion/exam-question/" + str(dataid) + "/"

    driver.get(url)
    sleep_random_sec(2)

    bs = BeautifulSoup(driver.page_source, 'html.parser')

    pattern_discuss_id = r' +data-discussion-id="([0-9]+)"'
    pattern_data_id = r' +data-linked-id="([0-9]+)"'
    pattern_title = r' +data-title="([^"]+) question ([0-9]+) discussion.*"'
    data_id = dataid
    discuss_id = 0
    match = re.search(pattern_discuss_id, str(bs))
    if match:
        discuss_id = int(match.group(1))
    else:
        match = re.search(pattern_data_id, str(bs))
        if match:
            data_id = int(match.group(1))
    match = re.search(pattern_title, str(bs))
    if match:
        q_title = match.group(1)
        q_number = int(match.group(2))

    print(f'{q_number}\t{discuss_id}\t{data_id}\t{q_title}')

    # data-discussion-id="18133"
    # data-linked-to="question"
    # data-linked-id=""
    # data-title="Exam AWS Certified Solutions Architect - Professional topic 2 question 141 discussion">

    # data-linked-to="question"
    # data-linked-id="818618"
    # data-title="Exam CISA topic 1 question 593 discussion.">

In [ ]:
driver = set_chrome_driver()


### Get Exam data_id

In [45]:
driver = set_chrome_driver()
driver.set_window_position(1800,10)

# getExamData(driver, 493149)
# getExamData(driver, 818618)

for i in range(100):
    getExamData(driver, i + 808661)

driver.close()

688	81343	808661	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
689	0	808662	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
690	84360	808663	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
691	81354	808664	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
692	81356	808665	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
693	90061	808666	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
694	83197	808667	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
695	90062	808668	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
696	84480	808669	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
697	82508	808670	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
698	87682	808671	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1
699	0	808672	Exam AWS Certified Solutions Architect - Associate SAA-C02 topic 1


### Get missing exam.

In [8]:
df = pd.read_csv('empty.csv', delimiter='\t', encoding='utf-8', header=None,
                names=['ExamNo', 'DiscussNo', 'DataNo', 'DiscussURL'],
                dtype={'ExamNo': int, 'DiscussNo': int, 'DataNo': int, 'DiscussURL': str},
                index_col=False)

driver = set_chrome_driver()
driver.set_window_position(1800,10)

for i in range(len(df))[:]:
    qid = int(df.at[i, 'ExamNo'])
    did = int(df.at[i, 'DiscussNo'])
    dataid = int(df.at[i, 'DataNo'])
    url = str(df.at[i, 'DiscussURL'])
    if did == 0:
        getExamData(driver, dataid)

driver.close()

593	0	818618	Exam CISA topic 1
595	0	818620	Exam CISA topic 1
596	0	818621	Exam CISA topic 1
597	0	818622	Exam CISA topic 1
599	0	818624	Exam CISA topic 1
602	0	818627	Exam CISA topic 1
603	0	818628	Exam CISA topic 1
606	0	818631	Exam CISA topic 1
608	0	818633	Exam CISA topic 1
610	0	818635	Exam CISA topic 1
611	0	818636	Exam CISA topic 1
614	0	818639	Exam CISA topic 1
615	0	818640	Exam CISA topic 1
616	0	818641	Exam CISA topic 1
617	0	818642	Exam CISA topic 1
618	0	818643	Exam CISA topic 1
620	0	818645	Exam CISA topic 1
621	0	818646	Exam CISA topic 1
623	0	818648	Exam CISA topic 1
628	0	818653	Exam CISA topic 1
629	0	818654	Exam CISA topic 1
630	0	818655	Exam CISA topic 1
632	0	818657	Exam CISA topic 1
635	0	818660	Exam CISA topic 1
636	0	818661	Exam CISA topic 1
639	0	818664	Exam CISA topic 1
640	0	818665	Exam CISA topic 1
642	0	818667	Exam CISA topic 1
643	0	818668	Exam CISA topic 1
644	0	818669	Exam CISA topic 1
645	0	818670	Exam CISA topic 1
646	0	818671	Exam CISA topic 1
647	0	81

### Merge CSV File

In [39]:
def merge_csv(fname1, fname2):
    df1 = pd.read_csv(fname1, delimiter='\t', encoding='utf-8', header=None,
                    names=['ExamName', 'ExamNo', 'DiscussNo', 'DataNo'],
                    dtype={'ExamName': str, 'ExamNo': int, 'DiscussNo': int, 'DataNo': int},
                    index_col=False)

    df2 = pd.read_csv(fname2, delimiter='\t', encoding='utf-8', header=None,
                    names=['ExamNo', 'DiscussNo', 'DiscussURL'],
                    dtype={'ExamNo': int, 'DiscussNo': int, 'DiscussURL': str},
                    index_col=False)

    merged_df = pd.merge(df1, df2, on=['ExamNo', 'DiscussNo'], how='outer')
    return merged_df

df = merge_csv('CISA.csv', 'CISA_Exam.csv')

df = df.sort_values(['ExamNo'], ascending=[True]).drop(columns=['ExamName'])

df
df.to_csv('CISA-1.csv', sep='\t', header=False, index=False)


In [36]:
def set_translate_to_kr2(driver):
    actionChains = ActionChains(driver)
    actionChains.context_click().perform()

    pyautogui.press('T')
    pyautogui.sleep(1)

def set_translate_to_kr(driver):
    actionChains = ActionChains(driver)
    actionChains.context_click().perform()

    # here maybe problem. Debug it:
    for i in range(3):
        pyautogui.press('up')

    pyautogui.press('enter')
    pyautogui.sleep(1)

def bring_to_top(driver):
    # Send ALT + TAB to switch to the browser window
    pyautogui.keyDown('alt')
    pyautogui.press('tab')
    pyautogui.keyUp('alt')

    # Optionally, you can use pyautogui.hotkey('alt', 'tab') to achieve the same effect

    # Bring the browser window to the top
    pyautogui.hotkey('win', 'up')  # On Windows
    # pyautogui.hotkey('command', 'up')  # On macOS

    # You can now interact with the browser
    pyautogui.press('enter')
    pyautogui.sleep(1)
    
    

bring_to_top(driver)

In [63]:
def set_translate_to_kr(driver):
    while True:
        # actionChains = ActionChains(driver)
        # actionChains.context_click().perform()
        pyautogui.hotkey('shift', 'f10')
        pyautogui.hotkey('T')
        time.sleep(1)

        pattern = r'>영어<'
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        match = re.search(pattern, str(bs))
        if match:
            return
        
        if platform.system() == "Windows":
            import winsound
            winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
        time.sleep(3)

        try:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
        except Exception as e:
            pass



In [58]:

driver = set_chrome_driver()
i=10


In [65]:
fn = f'CISM/CISM-Q{i+1:04}.html'

url = f'file:///Users/changwhaj/MyProjects/ExamTopics/assets/exam/isaca/{fn}'
url = f'file:///E:/MyProjects/ExamTopics/assets/exam/isaca/{fn}'
url = 'file:///E:/MyProjects/ExamTopics/assets/exam/isaca/CISA/TEMPLATE.html'

driver.get(url)
driver.switch_to.window(driver.window_handles[0])
time.sleep(1)
set_translate_to_kr(driver)
time.sleep(1)

bs = BeautifulSoup(driver.page_source, 'html.parser')


In [34]:
pattern = r'>영어<'
match = re.search(pattern, str(bs))
if match:
  print('true')
else:
  try:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
  except Exception as e:
    pass
  
  driver.switch_to.window(driver.window_handles[0])
  bring_to_top(driver)
  set_translate_to_kr(driver)


In [57]:
driver.close()
driver.quit()

In [10]:
if platform.system() == "Windows":
    import winsound
    winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)


In [9]:
driver.switch_to.window(driver.window_handles[0])

actionChains = ActionChains(driver)
actionChains.context_click().perform()
# actionChains.send_keys(Keys.ARROW_UP).perform()
# time.sleep(1)

# actionChains.send_keys(Keys.ARROW_UP).perform()

pyautogui.moveTo(x=150, y=350, duration=1)
pyautogui.click()

# actionChains.send_keys(Keys.ARROW_DOWN, Keys.ARROW_DOWN, Keys.RETURN).perform()


In [ ]:

context_menu_option = driver.find_element_by_xpath("//li[@class='context-menu-option']")
context_menu_option.click()

driver.switch_to.window(driver.window_handles[0]);

time.sleep(1)

actionChains.send_keys(Keys.ARROW_UP).perform()

pyautogui.moveTo(x=20, y=150, duration=1)
pyautogui.click()

actionChains.send_keys(Keys.ARROW_DOWN, Keys.ARROW_DOWN, Keys.RETURN).perform()




In [5]:
driver = set_chrome_driver()

for i in range(1196)[0:1]:

    fn = f'CISM/CISM-Q{i+1:04}.html'
    url = f'file:///Users/changwhaj/MyProjects/ExamTopics/assets/exam/{fn}'

    my_file = Path(fn)
    # file does not exists. Skip
    if not my_file.is_file(): continue

    driver.get(url)
    driver.switch_to.window(driver.window_handles[0])

    driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.reveal-solution').click()
    set_translate_to_kr(driver)
    driver.find_element(By.CSS_SELECTOR, 'a.btn.btn-primary.hide-solution').click()
    scroll_page(driver)
    try:
        driver.find_element(By.CSS_SELECTOR, '#scrollUp').click()
    except Exception as e:
        pass
    time.sleep(1)
    try:
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, '#goog-gt-tt'))
    except Exception as e:
        pass
    try:
        driver.execute_script("arguments[0].remove();", driver.find_element(By.CSS_SELECTOR, 'head > link'))
        driver.execute_script("arguments[0].removeAttribute('class');", driver.find_element(By.TAG_NAME, 'html'))
    except Exception as e:
        print(f"*** Error translate_page_to_kr !!! {e}")
        pass

    save_page(driver, fnkr)

driver.close()

Incompatible release of chromedriver (version 116.0.5845.96) detected in PATH: /usr/local/bin/chromedriver


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.110 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x00000001039ba6b8 chromedriver + 4937400
1   chromedriver                        0x00000001039b1b73 chromedriver + 4901747
2   chromedriver                        0x000000010356f616 chromedriver + 435734
3   chromedriver                        0x00000001035a1d10 chromedriver + 642320
4   chromedriver                        0x000000010359d18a chromedriver + 622986
5   chromedriver                        0x000000010359967c chromedriver + 607868
6   chromedriver                        0x00000001035e0a08 chromedriver + 899592
7   chromedriver                        0x00000001035dfebf chromedriver + 896703
8   chromedriver                        0x00000001035d6de3 chromedriver + 859619
9   chromedriver 